# preparing data

## 1. people

In [8]:
with open('names.txt', 'r', encoding='utf-8') as f:
    names = f.read()

In [11]:
names = names.split('\n')

## 2. books

In [12]:
!pip install beautifulsoup4
! pip install requests
! pip install urllib

You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.



  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
  Using cached soupsieve-2.3.1-py3-none-any.whl (37 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [13]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm

In [38]:
links = ['https://book24.ru/knigi-bestsellery/page-3/', 'https://book24.ru/knigi-bestsellery/page-4/', 
         'https://book24.ru/knigi-bestsellery/page-5/']

In [39]:
def get_authors(info):
    authors = []
    for book in info:
        inf = book.find_all('a', {'class':'author-list__item smartLink'})
        if len(inf) == 1:
            authors.append([inf[0].get_text()])
        elif len(inf) == 0:
            try:
                inf = book.find_all('span', {'class':'author-list__item'})
                authors.append([inf[0].get_text()])
            except:
                print(book)
        else:
            au = [i.get_text() for i in inf]
            authors.append(au)
    return authors

In [27]:
authors

[['Стивен Кинг'],
 ['Мия Шеридан'],
 ['Редгрейн Лебовски'],
 ['Эрин Уатт'],
 ['Шарлотта Бронте'],
 ['Аннетт Мари'],
 ['Харпер Ли'],
 ['Эрих Ремарк'],
 ['Лия Арден'],
 ['Аннетт Мари'],
 ['Холли Блэк'],
 ['Мадлен Миллер'],
 ['Ли Бардуго'],
 ['Керри Манискалко'],
 ['Элис Хоффман'],
 ['Стефани Гарбер'],
 ['Кристина Старк'],
 ['Лия Арден'],
 ['Мара Вульф'],
 ['Стивен Кинг'],
 ['Евгений Замятин'],
 ['Энтони Бёрджесс'],
 ['Редгрейн Лебовски'],
 ['Уильям Голдинг'],
 ['Скарлетт Сент-Клэр'],
 ['Фрэнсис Фицджеральд'],
 ['Донна Тартт'],
 ['Анна Джейн'],
 ['Олдос Хаксли'],
 ['Марисса Мейер']]

In [40]:
def get_book_names(soup):
    books = soup.find_all('a', {'class':'product-card__name smartLink'})
    books = [b.get_text().strip() for b in books]
    return books

In [80]:
def get_book_urls(soup):
    urls = []
    hrefs = soup.find_all('a', {'class':'product-card__image-link smartLink'})
    for href in hrefs:
        urls.append('https://book24.ru'+href['href'])
    return urls

In [81]:
# link = 'https://book24.ru/novie-knigi/'
books = []
authors = []
book_urls = []
for link in links:
    result = requests.get(link)
    html = result.text
    soup = BeautifulSoup(html,'html.parser')
    info = soup.find_all('div', {'class':'product-card__content'})
    book_urls.extend(get_book_urls(soup))
    authors.extend(get_authors(info))
    books.extend(get_book_names(soup))

In [42]:
len(authors)

90

In [43]:
len(books)

90

In [82]:
book_urls[:10]

['https://book24.ru/product/billi-sammers-6112249/',
 'https://book24.ru/product/archer-budet-molchat-5401079/',
 'https://book24.ru/product/kosti-i-zvezdnaya-pyl-5960605/',
 'https://book24.ru/product/slomlennyy-prints-1978304/',
 'https://book24.ru/product/dzheyn-eyr-207987/',
 'https://book24.ru/product/bessmertnyy-ogon-5428404/',
 'https://book24.ru/product/ubit-peresmeshnika-1792505/',
 'https://book24.ru/product/triumfalnaya-arka-1638121/',
 'https://book24.ru/product/mara-i-morok-5920467/',
 'https://book24.ru/product/temnaya-burya-5400356/']

In [62]:
for n, i in enumerate(info1):
    print(n, i.get_text())

0 Виктория Авеярд
1  Сергеева Валентина Сергеевна 
2 Young Adult. Мировой бестселлер Виктории Авеярд
3 Фантастика и фэнтези для подростков
4 Эксмо, Freedom
5  978-5-04-103445-0 
6  16+ 
7  2021 
8  544 
9  Твердый (7БЦ) 
10  Офсетная 
11  133x207  мм 
12  0.54  кг 


In [97]:
publishers = ['Эксмо, Freedom', 'АСТ', 'Like Book', 'Эксмо, Inspiria', 'Эксмо']

In [104]:
def get_book_info(book_link):
    book_info = {}
    result = requests.get(book_link)
    html = result.text
    soup = BeautifulSoup(html,'html.parser')
    info1 = soup.find_all('div', {'class':'product-characteristic__value'})
    for n, i in enumerate(info1):
        if n == 0:
            if len(i.get_text().split(',')) > 1:
                book_info['author'] = i.get_text().split(',')
            else:
                book_info['author'] = i.get_text()
        elif n == 3:
            if i.get_text() in publishers:
                book_info['genre'] = info1[2].get_text()
            else:
                book_info['genre'] = i.get_text()
        elif n == 7:
            if len(i.get_text().strip()) != 4:
                book_info['pub_year'] = info1[6].get_text().strip()
            else:
                book_info['pub_year'] = i.get_text().strip()
        
            
    return book_info

In [107]:
books_dict = {}
for book, url in tqdm(list(zip(books, book_urls))):
    books_dict[book] = get_book_info(url)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [02:15<00:00,  1.50s/it]


In [108]:
books_dict

{'Билли Саммерс': {'author': 'Стивен Кинг',
  'genre': 'Зарубежные детективы',
  'pub_year': '2022'},
 'Арчер будет молчать': {'author': 'Мия Шеридан',
  'genre': 'Зарубежная сентиментальная проза',
  'pub_year': '2019'},
 'Кости и Звездная пыль': {'author': 'Редгрейн Лебовски',
  'genre': 'Боевое фэнтези',
  'pub_year': '2021'},
 'Сломленный принц': {'author': 'Эрин Уатт',
  'genre': 'Зарубежная сентиментальная проза',
  'pub_year': '2022'},
 'Джейн Эйр': {'author': 'Шарлотта Бронте',
  'genre': 'Классическая зарубежная литература',
  'pub_year': '2022'},
 'Бессмертный огонь': {'author': 'Аннетт Мари',
  'genre': 'Зарубежное фэнтези',
  'pub_year': '2021'},
 'Убить пересмешника…': {'author': 'Харпер Ли',
  'genre': 'Классическая зарубежная литература',
  'pub_year': '2022'},
 'Триумфальная арка': {'author': 'Эрих Ремарк',
  'genre': 'Классическая зарубежная литература',
  'pub_year': '2021'},
 'Мара и Морок': {'author': 'Лия Арден',
  'genre': 'Любовное фэнтези',
  'pub_year': '2021'}

In [113]:
with open("books_info.json", "w", encoding='utf-8') as outfile:
    json.dump(books_dict, outfile, ensure_ascii=False)

In [114]:
genres = []
for book in books_dict.keys():
    genres.append(books_dict[book]['genre'])

In [116]:
genres

['Зарубежные детективы',
 'Зарубежная сентиментальная проза',
 'Боевое фэнтези',
 'Зарубежная сентиментальная проза',
 'Классическая зарубежная литература',
 'Зарубежное фэнтези',
 'Классическая зарубежная литература',
 'Классическая зарубежная литература',
 'Любовное фэнтези',
 'Зарубежное фэнтези',
 'Фантастика и фэнтези для подростков',
 'Современная зарубежная проза',
 'Зарубежное фэнтези',
 'Зарубежные детективы',
 'Зарубежное фэнтези',
 'Фантастика и фэнтези для подростков',
 'Современная русская проза',
 'Любовное фэнтези',
 'Фантастика и фэнтези для подростков',
 'Ужасы. Мистика',
 'Классическая русская литература',
 'Антиутопия',
 'Фантастика и фэнтези для подростков',
 'Классическая зарубежная литература',
 'Любовное фэнтези',
 'Классическая зарубежная литература',
 'Современная зарубежная проза',
 'Русская сентиментальная проза',
 'Антиутопия',
 'Космическая фантастика',
 'Фантастика и фэнтези для подростков',
 'Триллеры',
 'Зарубежное фэнтези',
 'Героическое фэнтези',
 'Вос

In [117]:
interests = ['Детективы', 'Фэнтези', 'Русская литература',
             'Зарубежная литература', 'Фантастика', 'Психология',
             'Политика', 'Ужасы', 'Романы', 'Кино']

In [118]:
with open("interests.txt", "w", encoding='utf-8') as f:
    f.write('\n'.join(interests))